In [1]:
from pathlib import Path
from pprint import pprint

# from darts import TimeSeries
# from darts.models.forecasting.arima import ARIMA
from statsmodels.tsa.arima.model import ARIMA as smARIMA
# from darts.models.forecasting.auto_arima import AutoARIMA
# from darts.models.forecasting.sf_auto_arima import StatsForecastAutoARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from darts.metrics.metrics import dtw_metric
from statsmodels.tsa.stattools import adfuller
# from mpire import WorkerPool
import pandas as pd

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [2]:
dfs_path = Path("../data/2_final/0__input/")

In [3]:
file_paths = sorted(dfs_path.iterdir())
N = len(file_paths)

In [4]:
def filter_df(df):
    return df.loc[df.train_set == 1, ["vals"]]

In [5]:
# # with WorkerPool(n_jobs=4) as pool:
# dfs = map(pd.read_parquet, file_paths)
# filtered_dfs = map(filter_df, dfs)
# adfuller_results = map(adfuller, filtered_dfs)
# final_results = list(map(lambda x: "ESTAC" if x[1] <= .05 else "NON ESTAC", adfuller_results))

In [6]:
# from datetime import datetime

# datetime.now()

In [7]:
# from collections import Counter
# Counter(final_results)

In [8]:
# print('ADF Statistic: {}'.format(result[0]))
# print('p-value: {}'.format(result[1]))
# print('Critical Values:')
# for key, value in result[4].items():
#     print('\t{}: {}'.format(key, value))

In [9]:
df = pd.read_parquet(file_paths[0]).reset_index().rename(columns={"index": "time"})

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79795 entries, 0 to 79794
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         79795 non-null  int64  
 1   vals         79795 non-null  float64
 2   train_set    79795 non-null  int64  
 3   test_set     79795 non-null  int64  
 4   anomaly_set  79795 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 3.0 MB


In [11]:
# series = TimeSeries.from_dataframe(df.loc[df.train_set == 1, ["time", "vals"]], time_col="time", value_cols="vals")

# series

In [12]:
# # model_autoarima = AutoARIMA(start_p=0, start_q=0, max_p=2, max_q=2, max_d=2, trace=True)
# model_arima = smARIMA(df.loc[df.train_set == 1, "vals"])

# # model_arima.residuals(series, retrain=False, verbose=True)

In [13]:
# model_arima.order

In [14]:
# result = model_arima.fit()
# result

In [15]:
# result.aic

In [16]:
# # order = (params['order_p'], params['order_d'], params['order_q'])
# # seasonal_order = (params['order_P'], params['order_D'], params['order_Q'], params['order_s'])
# # pprint(params)
# model_arima = SARIMAX(df.loc[df.train_set == 1, "vals"], trend='ct')#, order=order, seasonal_order=seasonal_order)
# result = model_arima.fit()
# # return result.aic

In [17]:
from pprint import pprint

25,5,21

In [26]:
def hyperopt_train_test(**params):
    order = (params['order_p'], params['order_d'], params['order_q'])
    # seasonal_order = (params['order_P'], params['order_D'], params['order_Q'], params['order_s'])
    pprint(params)
    model_arima = smARIMA(df.loc[df.train_set == 1, "vals"], order=order)
    # model_arima = SARIMAX(df.loc[df.train_set == 1, "vals"], order=order, seasonal_order=seasonal_order)
    result = model_arima.fit()
    return result.aic

param_space = {
    'order_p': hp.choice('order_p', range(31)),
    'order_d': hp.choice('order_d', range(31)),
    'order_q': hp.choice('order_q', range(31)),
    # 'order_P': hp.choice('order_P', range(30)),
    # 'order_D': hp.choice('order_D', range(30)),
    # 'order_Q': hp.choice('order_Q', range(30)),
    # 'order_s': hp.choice('order_s', range(30)),
}

def f(params):
    acc = hyperopt_train_test(**params)
    return {'loss': acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=100, trials=trials)
print("best", best)

{'order_d': 14, 'order_p': 3, 'order_q': 19}                                                                                                                                                                       
{'order_d': 24, 'order_p': 18, 'order_q': 10}                                                                                                                                                                      
  1%|█▎                                                                                                                                       | 1/100 [15:47<26:03:22, 947.50s/trial, best loss: 506330.0370499939]

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



{'order_d': 16, 'order_p': 30, 'order_q': 11}                                                                                                                                                                      
  2%|██▉                                                                                                                                                  | 2/100 [44:26<38:08:33, 1401.15s/trial, best loss: 58.0]

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



{'order_d': 2, 'order_p': 24, 'order_q': 19}                                                                                                                                                                       
  3%|████▍                                                                                                                                              | 3/100 [1:37:01<59:19:48, 2201.94s/trial, best loss: 58.0]

...

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'



{'order_d': 25, 'order_p': 7, 'order_q': 7}                                                                                                                                                                        
 18%|██████████████████████████▎                                                                                                                       | 18/100 [6:22:18<36:32:43, 1604.43s/trial, best loss: 58.0]

/home/paulo/.pyenv/versions/3.10.5/envs/tcc/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'

job exception: LU decomposition error.



 18%|██████████████████████████▎                                                                                                                       | 18/100 [6:31:25<29:43:10, 1304.76s/trial, best loss: 58.0]


LinAlgError: LU decomposition error.

In [ ]:
# from flaml import AutoML

# automl = AutoML()

# settings = {
#     # "time_budget": 10,  # total running time in seconds
#     "metric": dtw_metric,  # primary metric for validation: 'mape' is generally used for forecast tasks
#     "task": 'ts_forecast',  # task type
#     "log_file_name": 'automl.log',  # flaml log file
#     "eval_method": "holdout",  # validation method can be chosen from ['auto', 'holdout', 'cv']
#     "seed": 7654321,  # random seed
#     "log_training_metric": True,
# #     'order_p': tune.randint(lower=1, upper=100000),
# #     'order_d': tune.randint(lower=1, upper=100000),
# #     'order_q': tune.randint(lower=1, upper=100000),
# }

# automl.fit(
#     dataframe=df.loc[df.train_set == 1, ["time", "vals"]],  # training data
#     label='vals',  # label column
#     period=1,  # key word argument 'period' must be included for forecast task)
#     **settings
# )

# # def f(params):
# #     acc = hyperopt_train_test(**params)
# #     return {'loss': -acc, 'status': STATUS_OK}

# # trials = Trials()
# # best = fmin(f, param_space, algo=tpe.suggest, max_evals=100, trials=trials)
# # print("best", best)